In [1]:
!pip install pandas
!pip install extract-msg
!pip install openpyxl
!pip install pywin32
!pip install beautifulsoup4
!pip install XlsxWriter
!pip install tabulate

ERROR: Could not find a version that satisfies the requirement pywin32 (from versions: none)
ERROR: No matching distribution found for pywin32


## DESCRIPCION : 

El siguiente codigo se encarga de buscar en el correo del estudiante de practicas todos los correos que tengan como asunto las palabras , Veeam Backup y Veeam y Veam y aparte pida el mes ya que los informes son por meses entonces el asunto siempre va a tener alguna de las palabras que se mencionan anteiormente y organizarlos en diferentes hojas y columnas segun corresponda.

# MI PRUEBA DESDE 0

In [1]:
from datetime import datetime
import pytz
import os
import unicodedata
import win32com.client
import email
from email.header import decode_header
import email.utils

# === Funciones auxiliares ===
def normalizar(texto):
    return ''.join(c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')

def leer_eml(archivo, archivo_salida):
    try:
        with open(archivo, 'r', encoding='utf-8') as f:
            mensaje_eml = email.message_from_file(f)

            asunto, encoding = decode_header(mensaje_eml["Subject"])[0]
            if isinstance(asunto, bytes):
                asunto = asunto.decode(encoding or 'utf-8')
            print(f"   ✉️ Asunto del .eml: {asunto}")
            archivo_salida.write(f"   ✉️ Asunto del .eml: {asunto}\n")

            received_headers = mensaje_eml.get_all("Received")
            fecha_objetivo = None

            if received_headers:
                for line in reversed(received_headers):
                    if ";" in line:
                        raw_fecha = line.split(";")[-1].strip()
                        if "-0500" in raw_fecha and "(-05)" in raw_fecha:
                            parsed = email.utils.parsedate_tz(raw_fecha)
                            if parsed:
                                timestamp = email.utils.mktime_tz(parsed)
                                dt = datetime.fromtimestamp(timestamp, pytz.timezone("America/Bogota"))
                                fecha_objetivo = dt.strftime('%a %Y-%m-%d %H:%M')
                                break

            if fecha_objetivo:
                print(f"   🕒 Hora real de recibido en COT: {fecha_objetivo}")
                archivo_salida.write(f"   🕒 Hora real de recibido en COT: {fecha_objetivo}\n")
            else:
                print("   ⚠️ No se encontró una fecha con zona -0500 (-05).")
                archivo_salida.write("   ⚠️ No se encontró una fecha con zona -0500 (-05).\n")

            cuerpo = ""
            if mensaje_eml.is_multipart():
                for parte in mensaje_eml.walk():
                    ctype = parte.get_content_type()
                    cdispo = str(parte.get("Content-Disposition"))
                    if ctype == "text/plain" and "attachment" not in cdispo:
                        charset = parte.get_content_charset() or "utf-8"
                        cuerpo = parte.get_payload(decode=True).decode(charset, errors="ignore")
                        break
            else:
                charset = mensaje_eml.get_content_charset() or "utf-8"
                cuerpo = mensaje_eml.get_payload(decode=True).decode(charset, errors="ignore")

            if cuerpo:
                print("   🧾 Cuerpo del correo:")
                print(cuerpo)
                archivo_salida.write("   🧾 Cuerpo del correo:\n")
                archivo_salida.write(cuerpo + "\n")
            else:
                print("   ⚠️ No se encontró cuerpo legible.")
                archivo_salida.write("   ⚠️ No se encontró cuerpo legible.\n")

    except Exception as e:
        print(f"⚠️ Error al leer .eml: {e}")
        archivo_salida.write(f"⚠️ Error al leer .eml: {e}\n")

# === Inputs ===
mes = input("Ingresa el mes (Ej: Abril): ").capitalize()
anio = input("Ingresa el año (Ej: 2025): ")
dia_inicio = int(input("Día de inicio (Ej: 1): "))
dia_fin = int(input("Día de fin (Ej: 31): "))

meses_esp = {
    "Enero": "January", "Febrero": "February", "Marzo": "March", "Abril": "April",
    "Mayo": "May", "Junio": "June", "Julio": "July", "Agosto": "August",
    "Septiembre": "September", "Octubre": "October", "Noviembre": "November", "Diciembre": "December"
}
mes_num = list(meses_esp.keys()).index(mes) + 1
colombia_tz = pytz.timezone("America/Bogota")

# === Outlook ===
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
buzon = outlook.Folders.Item("epseguridadti@olimpica.com.co")
carpeta = buzon.Folders["Veam BK"]
mensajes = carpeta.Items
mensajes.Sort("[ReceivedTime]", True)

# === A partir de aquí, también se guarda en salida.txt ===
print("\n📨 Correos encontrados:")

with open("salida.txt", "w", encoding="utf-8") as archivo_salida:
    archivo_salida.write("📨 Correos encontrados:\n")

    for mensaje in mensajes:
        try:
            asunto = normalizar(mensaje.Subject)
            recibido = mensaje.ReceivedTime

            if "Veeam" in asunto and mes in asunto and anio in asunto and dia_inicio <= recibido.day <= dia_fin and recibido.month == mes_num:
                recibido_colombia = recibido.astimezone(colombia_tz)

                print(f"\n🕒 Recibido: {recibido_colombia.strftime('%d/%m/%Y %H:%M')}")
                print(f"Asunto: {mensaje.Subject}")
                archivo_salida.write(f"\n🕒 Recibido: {recibido_colombia.strftime('%d/%m/%Y %H:%M')}\n")
                archivo_salida.write(f"Asunto: {mensaje.Subject}\n")

                if mensaje.Attachments.Count > 0:
                    print("📎 Adjuntos encontrados:")
                    archivo_salida.write("📎 Adjuntos encontrados:\n")
                    for i in range(1, mensaje.Attachments.Count + 1):
                        adjunto = mensaje.Attachments.Item(i)
                        nombre = adjunto.FileName
                        print(f" - {nombre}")
                        archivo_salida.write(f" - {nombre}\n")

                        if nombre.lower().endswith(".eml"):
                            print("   🔍 Analizando contenido del .eml...")
                            archivo_salida.write("   🔍 Analizando contenido del .eml...\n")

                            carpeta_destino = os.path.join(os.getcwd(), "Archivos_eml")
                            if not os.path.exists(carpeta_destino):
                                os.makedirs(carpeta_destino)

                            ruta_temp = os.path.join(carpeta_destino, nombre)
                            adjunto.SaveAsFile(ruta_temp)

                            leer_eml(ruta_temp, archivo_salida)

                            os.remove(ruta_temp)
                            print(f"🗑️ Archivo {nombre} eliminado.")
                            archivo_salida.write(f"🗑️ Archivo {nombre} eliminado.\n")
        except Exception as e:
            print(f"⚠️ Error leyendo mensaje: {e}")
            archivo_salida.write(f"⚠️ Error leyendo mensaje: {e}\n")


Ingresa el mes (Ej: Abril): mayo
Ingresa el año (Ej: 2025): 2025
Día de inicio (Ej: 1): 1
Día de fin (Ej: 31): 31

📨 Correos encontrados:

🕒 Recibido: 23/05/2025 03:52
Asunto: FW: Informe Final Veeam Backup - Olímpica S.A. 21 de Mayo 2025
📎 Adjuntos encontrados:
 - Informe Final Veeam Backup - Olímpica S.A. 21 de Mayo 2025.eml
   🔍 Analizando contenido del .eml...
   ✉️ Asunto del .eml: Informe Final Veeam Backup - Olímpica S.A. 21 de Mayo 2025
   🕒 Hora real de recibido en COT: Thu 2025-05-22 22:20
   🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.


Replication
Observaciones
Porcentaje completado
71%
FINALIZADO
Total de Jobs Programados
31
Jobs Replicados
22
Jobs No Replicados
9
Hora de inicio de la ventana
22/05/2025 - 12:00 AM
Hora de finalización de la ventana
                 22/05/2025 - 9:03 PM
Tiempo de backup
                                        21h 03m


NOTA: Con el 

   ✉️ Asunto del .eml: Informe Veeam Backup - Olímpica S.A. 21 de Mayo 2025
   🕒 Hora real de recibido en COT: Thu 2025-05-22 16:25
   🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.
Backup a disco - veeam
Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
Cantidad MV respaldadas de Jobs ejecutados
200
Cantidad MV no respaldadas de los Jobs ejecutados
0
Hora de inicio de la ventana
21/5/2025 - 7:00 PM
Hora de finalización de la ventana
21/05/2025 - 10:41 PM
Tiempo de Backup
3h 41m
Backup a cinta - TAPE_OLIMPICA_NEW
  Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
TB respaldadas
66.4 TB
TB no respaldadas
0 TB
Hora de inicio
21/05/2025 - 10:46 PM
Hora de finalización
22/05/2025 - 3:52 PM
Tiempo de Backup
17h 05m
Estado
FINALIZADO
Replication
Observaciones
Porcentaje completado
56%
EN EJECUCIÓN
Total de Jobs Programados
31
Jobs Replica

   ✉️ Asunto del .eml: Informe Veeam Backup - Olímpica S.A. 21 de Mayo 2025
   🕒 Hora real de recibido en COT: Thu 2025-05-22 10:51
   🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.
Backup a disco - veeam
Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
Cantidad MV respaldadas de Jobs ejecutados
200
Cantidad MV no respaldadas de los Jobs ejecutados
0
Hora de inicio de la ventana
21/5/2025 - 7:00 PM
Hora de finalización de la ventana
21/05/2025 - 10:41 PM
Tiempo de Backup
3h 41m
Backup a cinta - TAPE_OLIMPICA_NEW
  Observaciones
Porcentaje completado
96%
ESPERANDO POR CINTA
Porcentaje faltante
4%
TB respaldadas
66.2 TB
TB no respaldadas
2.2 TB
Hora de inicio
21/05/2025 - 10:46 PM
Hora de finalización

Tiempo de Backup
11h 50m
Estado
EN EJECUCIÓN
Replication
Observaciones
Porcentaje completado
56%
EN EJECUCIÓN
Total de Jobs Programados
31
Jobs Replicados
17
J

 - Informe Final Veeam Backup - Olímpica S.A. 20 de Mayo 2025.eml
   🔍 Analizando contenido del .eml...
   ✉️ Asunto del .eml: Informe Final Veeam Backup - Olímpica S.A. 20 de Mayo 2025
   🕒 Hora real de recibido en COT: Thu 2025-05-22 06:16
   🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.


Replication
Observaciones
Porcentaje completado
99%
FINALIZADO
Total de Jobs Programados
31
Jobs Replicados
30
Jobs No Replicados
1
Hora de inicio de la ventana
21/05/2025 - 12:00 AM
Hora de finalización de la ventana
     21/05/2025 - 11:11 PM
Tiempo de backup
                              23h 11m



Quedamos atentos a sus comentarios.


Siéntase en libertad de comunicarse con nosotros a nuestra línea en Barranquilla tel. 3612752, a nuestra línea nacional 018000119966, o a nuestro correo electrónico servicio@redsis.com<mailto:servicio@redsis.com>, donde le brindaremos la información adicional


🕒 Recibido: 22/05/2025 03:13
Asunto: FW: Informe Veeam Backup - Olímpica S.A. 20 de Mayo 2025
📎 Adjuntos encontrados:
 - Informe Veeam Backup - Olímpica S.A. 20 de Mayo 2025.eml
   🔍 Analizando contenido del .eml...
   ✉️ Asunto del .eml: Informe Veeam Backup - Olímpica S.A. 20 de Mayo 2025
   🕒 Hora real de recibido en COT: Wed 2025-05-21 16:59
   🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.

Backup a disco - veeam
Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
Cantidad MV respaldadas de Jobs ejecutados
200
Cantidad MV no respaldadas de los Jobs ejecutados
0
Hora de inicio de la ventana
20/5/2025 - 7:00 PM
Hora de finalización de la ventana
20/05/2025 - 10:27 PM
Tiempo de Backup
3h 27m
Backup a cinta - TAPE_OLIMPICA_NEW
  Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
TB respaldadas
66.3 TB
TB no respaldadas
0 TB
Hora de 

...

   ✉️ Asunto del .eml: Informe Veeam Backup - Olímpica S.A. 3 de Mayo 2025
   🕒 Hora real de recibido en COT: Sun 2025-05-04 19:23
   🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.

Backup a disco - veeam
Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
Cantidad MV respaldadas de Jobs ejecutados
200
Cantidad MV no respaldadas de los Jobs ejecutados
0
Hora de inicio de la ventana
3/5/2025 - 7:00 PM
Hora de finalización de la ventana
         4/5/2025 - 1:11 AM
Tiempo de Backup
6h 11m
Backup a cinta - TAPE_OLIMPICA_NEW
  Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
TB respaldadas
65.7 TB
TB no respaldadas
0 TB
Hora de inicio
      4/5/2025 - 1:11 AM
Hora de finalización
4/5/2025 - 1:27 PM
Tiempo de Backup
          12h 15m
Estado
FINALIZADO
Replication
Observaciones
Porcentaje completado
100%
FINALIZADO
Total de Jobs Programados


   ✉️ Asunto del .eml: Informe Final Veeam Backup - Olímpica S.A. 2 de Mayo 2025
   🕒 Hora real de recibido en COT: Sun 2025-05-04 00:44
   🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.

Backup a disco - veeam
Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
Cantidad MV respaldadas de Jobs ejecutados
200
Cantidad MV no respaldadas de los Jobs ejecutados
0
Hora de inicio de la ventana
2/5/2025 - 7:00 PM
Hora de finalización de la ventana
         2/5/2025 - 11:48 PM
Tiempo de Backup
4h 48m
Backup a cinta - TAPE_OLIMPICA_NEW
  Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
TB respaldadas
65.6 TB
TB no respaldadas
0 TB
Hora de inicio
      2/5/2025 - 11:48 PM
Hora de finalización
3/5/2025 - 12:01 PM
Tiempo de Backup
          12h 12m
Estado
FINALIZADO
Replication
Observaciones
Porcentaje completado
99%
FINALIZADO
Total de Jobs Prog

   ✉️ Asunto del .eml: Informe Veeam Backup - Olímpica S.A. 2 de Mayo 2025
   🕒 Hora real de recibido en COT: Sat 2025-05-03 11:01
   🧾 Cuerpo del correo:
Cordial saludo, estimado cliente

El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.
Backup a disco - veeam
Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
Cantidad MV respaldadas de Jobs ejecutados
200
Cantidad MV no respaldadas de los Jobs ejecutados
0
Hora de inicio de la ventana
2/5/2025 - 7:00 PM
Hora de finalización de la ventana
         2/5/2025 - 11:48 PM
Tiempo de Backup
4h 48m
Backup a cinta - TAPE_OLIMPICA_MENSUAL
  Observaciones
Porcentaje completado
98%
EN EJECUCIÓN
Porcentaje faltante
2%
TB respaldadas
64.5 TB
TB no respaldadas
1.1 TB
Hora de inicio
      2/5/2025 - 11:48 PM
Hora de finalización

Tiempo de Backup
          10h 59m
Estado
RUNNING
Replication
Observaciones
Porcentaje completado
52%
EN EJECUCIÓN
Total de Jobs Programados
31
Jobs Repli

   ✉️ Asunto del .eml: Informe Final Veeam Backup - Olímpica S.A. 1 de Mayo 2025
   🕒 Hora real de recibido en COT: Fri 2025-05-02 19:06
   🧾 Cuerpo del correo:
Cordial saludo, estimado cliente

El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.


Replication
Observaciones
Porcentaje completado
98%
FINALIZADO
Total de Jobs Programados
31
Jobs Replicados
30
Jobs No Replicados
1
Hora de inicio de la ventana
2/5/2025 - 12:00 AM
Hora de finalización de la ventana
2/5/2025 - 6:15 PM
Tiempo de backup
                       18h 15m


Quedamos atentos a sus comentarios.


Siéntase en libertad de comunicarse con nosotros a nuestra línea en Barranquilla tel. 3612752, a nuestra línea nacional 018000119966, o a nuestro correo electrónico servicio@redsis.com<mailto:servicio@redsis.com>, donde le brindaremos la información adicional que pueda requerir.



Es un gusto poder atenderle.





[cid:0feee93a-f2fc-4ef5-9875-a327e359238d]

Centro de Ayuda a Clien

   ✉️ Asunto del .eml: Informe Final Mensual Veeam Backup - Olímpica S.A. 30 de Abril 2025
   🕒 Hora real de recibido en COT: Fri 2025-05-02 05:23
   🧾 Cuerpo del correo:
Cordial saludo, estimado cliente.


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.

Replication
Observaciones
Porcentaje completado
99%
FINALIZADO
Total de Jobs Programados
31
Jobs Replicados
30
Jobs No Replicados
1
Hora de inicio de la ventana
1/5/2025 - 01:45 AM
Hora de finalización de la ventana
   2/5/2025 - 2:06 AM
Tiempo de backup
                       24h 21m



Quedamos atentos a sus comentarios.


Siéntase en libertad de comunicarse con nosotros a nuestra línea en Barranquilla tel. 3612752, a nuestra línea nacional 018000119966, o a nuestro correo electrónico servicio@redsis.com<mailto:servicio@redsis.com>, donde le brindaremos la información adicional que pueda requerir.



Es un gusto poder atenderle.





[cid:48514d4c-5a26-4534-bdcc-2914a395d92b]

Centro d

## ORGANIZAR INFORMACION DE CUERPO DE CORREOS SALIDA.TXT

# PRUEBA DE GEMINI 

In [41]:
import re
import pandas as pd
from datetime import datetime
import pytz
import os
import unicodedata
import win32com.client
import email
from email.header import decode_header
import email.utils

# --- Funciones auxiliares ---
def normalizar(texto):
    """Normaliza un texto quitando tildes y caracteres especiales."""
    return ''.join(c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')

def round_to_nearest_minute(dt_obj):
    """
    Redondea un objeto datetime al minuto más cercano.
    Si los segundos son 30 o más, redondea al siguiente minuto.
    Si los segundos son menos de 30, trunca los segundos al minuto actual.
    """
    if dt_obj.second >= 30:
        # Sube al siguiente minuto y elimina segundos y microsegundos
        dt_obj = dt_obj.replace(second=0, microsecond=0) + pd.Timedelta(minutes=1)
    else:
        # Trunca los segundos y microsegundos
        dt_obj = dt_obj.replace(second=0, microsecond=0)
    return dt_obj

def leer_eml(archivo, archivo_salida, colombia_tz):
    """
    Lee un archivo .eml, extrae información relevante como asunto,
    hora de envío y cuerpo, y la escribe en el archivo de salida.
    """
    try:
        with open(archivo, 'r', encoding='utf-8') as f:
            mensaje_eml = email.message_from_file(f)

            # --- Asunto del EML adjunto ---
            asunto, encoding = decode_header(mensaje_eml["Subject"])[0]
            if isinstance(asunto, bytes):
                asunto = asunto.decode(encoding or 'utf-8', errors='ignore')
            print(f"    ✉️ Asunto del .eml: {asunto}")
            archivo_salida.write(f"    ✉️ Asunto del .eml: {asunto}\n")

            # --- ANALISIS DEL ENCABEZADO "Date" (Hora de Envío Original del EML) ---
            fecha_eml_obj_date = None
            if "Date" in mensaje_eml:
                try:
                    parsed_date_from_header = email.utils.parsedate_to_datetime(mensaje_eml["Date"])

                    # Siempre convertir a la zona de Bogotá
                    if parsed_date_from_header.tzinfo is None:
                        # Si es naive, lo asumimos como UTC y lo convertimos a la zona horaria de Colombia
                        fecha_eml_obj_date = pytz.utc.localize(parsed_date_from_header).astimezone(colombia_tz)
                    else:
                        # Si ya tiene tzinfo, solo lo convertimos a la zona horaria de Colombia
                        fecha_eml_obj_date = parsed_date_from_header.astimezone(colombia_tz)

                except Exception as e:
                    print(f"    ⚠️ Error al parsear 'Date' header de EML adjunto: {e}")
                    archivo_salida.write(f"    ⚠️ Error al parsear 'Date' header de EML adjunto: {e}\n")
            else:
                print("    ⚠️ No se encontró 'Date' header en el EML adjunto.")
                archivo_salida.write("    ⚠️ No se encontró 'Date' header en el EML adjunto.\n")

            # --- ANALISIS Y SELECCIÓN DE HORA DE RECEPCIÓN (Received Headers del EML) ---
            received_headers = mensaje_eml.get_all("Received")
            all_parsed_received_dates_in_cot = []

            if received_headers:
                for header_raw in received_headers:
                    try:
                        date_part = header_raw.split(';')[-1].strip()
                        parsed_received_date = email.utils.parsedate_to_datetime(date_part)

                        if parsed_received_date:
                            if parsed_received_date.tzinfo is None:
                                # Si el received header no tiene TZ, asumimos UTC y convertimos a COT
                                received_date_in_bogota = pytz.utc.localize(parsed_received_date).astimezone(colombia_tz)
                            else:
                                # Si ya tiene tzinfo, lo convertimos a COT
                                received_date_in_bogota = parsed_received_date.astimezone(colombia_tz)

                            all_parsed_received_dates_in_cot.append(received_date_in_bogota)

                    except Exception:
                        pass # Silenciar errores de Received headers individuales
            
            # Determinar la hora final del EML, priorizando el Received más reciente y redondeando
            final_eml_time = None
            if all_parsed_received_dates_in_cot:
                latest_received_date_for_display = max(all_parsed_received_dates_in_cot)
                final_eml_time = round_to_nearest_minute(latest_received_date_for_display)
            elif fecha_eml_obj_date:
                final_eml_time = round_to_nearest_minute(fecha_eml_obj_date)

            if final_eml_time:
                print(f"    🕒 Hora de envío (EML Adjunto en COT): {final_eml_time.strftime('%a %Y-%m-%d %H:%M')}")
                archivo_salida.write(f"    🕒 Hora de envío (EML Adjunto en COT): {final_eml_time.strftime('%a %Y-%m-%d %H:%M')}\n")
            else:
                print("    ⚠️ No se pudo determinar la hora de envío del EML adjunto.")
                archivo_salida.write("    ⚠️ No se pudo determinar la hora de envío del EML adjunto.\n")

            # --- Cuerpo del correo ---
            cuerpo = ""
            if mensaje_eml.is_multipart():
                for parte in mensaje_eml.walk():
                    ctype = parte.get_content_type()
                    cdispo = str(parte.get("Content-Disposition"))
                    if ctype == "text/plain" and "attachment" not in cdispo:
                        charset = parte.get_content_charset() or "utf-8"
                        cuerpo = parte.get_payload(decode=True).decode(charset, errors="ignore")
                        break
            else:
                charset = mensaje_eml.get_content_charset() or "utf-8"
                cuerpo = mensaje_eml.get_payload(decode=True).decode(charset, errors="ignore")

            if cuerpo:
                print("    🧾 Cuerpo del correo:")
                print(cuerpo)
                archivo_salida.write("    🧾 Cuerpo del correo:\n")
                archivo_salida.write(cuerpo + "\n")
            else:
                print("    ⚠️ No se encontró cuerpo legible.")
                archivo_salida.write("    ⚠️ No se encontró cuerpo legible.\n")

    except Exception as e:
        print(f"⚠️ Error al leer .eml: {e}")
        archivo_salida.write(f"⚠️ Error al leer .eml: {e}\n")

# --- Inputs del usuario ---
mes = input("Ingresa el mes (Ej: Abril): ").capitalize()
anio = input("Ingresa el año (Ej: 2025): ")
dia_inicio = int(input("Día de inicio (Ej: 1): "))
dia_fin = int(input("Día de fin (Ej: 31): "))

# --- Configuración de zona horaria y mapeo de meses ---
meses_esp = {
    "Enero": "January", "Febrero": "February", "Marzo": "March", "Abril": "April",
    "Mayo": "May", "Junio": "June", "Julio": "July", "Agosto": "August",
    "Septiembre": "September", "Octubre": "October", "Noviembre": "November", "Diciembre": "December"
}
mes_num = list(meses_esp.keys()).index(mes) + 1
colombia_tz = pytz.timezone("America/Bogota")

# --- Conexión a Outlook ---
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
try:
    buzon = outlook.Folders.Item("epseguridadti@olimpica.com.co")
except Exception:
    print("No se encontró el buzón 'epseguridadti@olimpica.com.co'. Intentando usar el buzón por defecto...")
    buzon = outlook.Session.DefaultStore.GetRootFolder()

# --- Acceso a la carpeta y ordenamiento de mensajes ---
carpeta = buzon.Folders["Veam BK"]
mensajes = carpeta.Items
mensajes.Sort("[ReceivedTime]", True) # Ordena por fecha de recepción, descendente

# --- Procesamiento de correos y escritura en salida.txt ---
print("\n📨 Correos encontrados:")

with open("salida.txt", "w", encoding="utf-8") as archivo_salida:
    archivo_salida.write("📨 Correos encontrados:\n")

    for mensaje in mensajes:
        try:
            asunto = normalizar(mensaje.Subject)

            raw_received_time_outlook = mensaje.ReceivedTime

            # Convertir la hora de recepción de Outlook a la zona horaria de Bogotá y redondear
            if raw_received_time_outlook.tzinfo is not None:
                # Si tiene tzinfo, lo hacemos naive primero para evitar doble conversión o errores
                received_as_naive = raw_received_time_outlook.replace(tzinfo=None)
            else:
                received_as_naive = raw_received_time_outlook # Ya era naive

            # Localizamos esta hora naive directamente a la zona horaria de Bogotá.
            recibido_localized = colombia_tz.localize(received_as_naive)

            # Redondear el ReceivedTime del correo principal al minuto más cercano
            recibido = round_to_nearest_minute(recibido_localized)

            # --- Filtrar por fecha y asunto ---
            if "Veeam" in asunto and mes in asunto and anio in asunto and \
               dia_inicio <= recibido.day <= dia_fin and recibido.month == mes_num:

                fecha_hora_cot_str = recibido.strftime('%d/%m/%Y %H:%M')

                print(f"\n--- Procesando correo ---")
                print(f"🕒 Recibido (Outlook COT): {fecha_hora_cot_str}")
                print(f"Asunto: {mensaje.Subject}")
                archivo_salida.write(f"\n--- Procesando correo ---\n")
                archivo_salida.write(f"🕒 Recibido: {fecha_hora_cot_str}\n")
                archivo_salida.write(f"Asunto: {mensaje.Subject}\n")

                if mensaje.Attachments.Count > 0:
                    print("📎 Adjuntos encontrados:")
                    archivo_salida.write("📎 Adjuntos encontrados:\n")
                    for i in range(1, mensaje.Attachments.Count + 1):
                        adjunto = mensaje.Attachments.Item(i)
                        nombre = adjunto.FileName
                        print(f" - {nombre}")
                        archivo_salida.write(f" - {nombre}\n")

                        if nombre.lower().endswith(".eml"):
                            print("    🔍 Analizando contenido del .eml...")
                            archivo_salida.write("    🔍 Analizando contenido del .eml...\n")

                            carpeta_destino = os.path.join(os.getcwd(), "Archivos_eml")
                            if not os.path.exists(carpeta_destino):
                                os.makedirs(carpeta_destino)

                            ruta_temp = os.path.join(carpeta_destino, nombre)
                            adjunto.SaveAsFile(ruta_temp)

                            leer_eml(ruta_temp, archivo_salida, colombia_tz) # Pasamos colombia_tz

                            os.remove(ruta_temp)
                            print(f"🗑️ Archivo {nombre} eliminado.")
                            archivo_salida.write(f"🗑️ Archivo {nombre} eliminado.\n")
                else:
                    # Si no hay adjuntos EML, procesa el cuerpo del correo principal
                    cuerpo_principal = mensaje.Body
                    if cuerpo_principal:
                        print("    🧾 Cuerpo del correo principal:")
                        # Limita la impresión a 500 caracteres para no saturar la consola
                        print(cuerpo_principal[:500] + "..." if len(cuerpo_principal) > 500 else cuerpo_principal)
                        archivo_salida.write("    🧾 Cuerpo del correo principal:\n")
                        archivo_salida.write(cuerpo_principal + "\n")
                    else:
                        print("    ⚠️ No se encontró cuerpo legible en el correo principal.")
                        archivo_salida.write("    ⚠️ No se encontró cuerpo legible en el correo principal.\n")
                    # Aunque no se elimine un archivo, se registra que se procesó
                    archivo_salida.write(f"🗑️ Archivo (sin adjunto .eml) procesado para el asunto: {mensaje.Subject}.\n")
        except Exception as e:
            print(f"⚠️ Error procesando mensaje: {e}")
            archivo_salida.write(f"⚠️ Error procesando mensaje: {e}\n")

print("\nProcesamiento de correos finalizado.")

Ingresa el mes (Ej: Abril): Mayo
Ingresa el año (Ej: 2025): 2025
Día de inicio (Ej: 1): 1
Día de fin (Ej: 31): 31

📨 Correos encontrados:

--- Procesando correo ---
🕒 Recibido (Outlook COT): 28/05/2025 07:58
Asunto: FW: Informe Veeam Backup - Olímpica S.A. 26 de Mayo 2025
📎 Adjuntos encontrados:
 - Informe Veeam Backup - Olímpica S.A. 26 de Mayo 2025.eml
    🔍 Analizando contenido del .eml...
    ✉️ Asunto del .eml: Informe Veeam Backup - Olímpica S.A. 26 de Mayo 2025
    🕒 Hora de envío (EML Adjunto en COT): Tue 2025-05-27 22:58
    🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.

Backup a disco - veeam
Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
Cantidad MV respaldadas de Jobs ejecutados
200
Cantidad MV no respaldadas de los Jobs ejecutados
0
Hora de inicio de la ventana
26/5/2025 - 7:00 PM
Hora de finalización de la ventana
27/05/2025 - 4:17 AM
Tie

    ✉️ Asunto del .eml: Informe Veeam Backup - Olímpica S.A. 26 de Mayo 2025
    🕒 Hora de envío (EML Adjunto en COT): Tue 2025-05-27 06:01
    🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.

Backup a disco - veeam
Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
Cantidad MV respaldadas de Jobs ejecutados
200
Cantidad MV no respaldadas de los Jobs ejecutados
0
Hora de inicio de la ventana
26/5/2025 - 7:00 PM
Hora de finalización de la ventana
27/05/2025 - 4:17 AM
Tiempo de Backup
9h 17m
Backup a cinta - TAPE_OLIMPICA_NEW
  Observaciones
Porcentaje completado
69%
EN EJECUCIÓN
Porcentaje faltante
31%
TB respaldadas
36 TB
TB no respaldadas
15.4 TB
Hora de inicio
26/05/2025 - 11:25 PM
Hora de finalización

Tiempo de Backup
6h 16m
Estado
EN EJECUCIÓN
Replication
Observaciones
Porcentaje completado
48%
EN EJECUCIÓN
Total de Jobs Programados
31
Jobs Replicados
15


🗑️ Archivo Informe FInal Veeam Backup - Olímpica S.A. 25 de Mayo 2025.eml eliminado.
 - Informe Veeam Backup - Olímpica S.A. 25 de Mayo 2025.eml
    🔍 Analizando contenido del .eml...
    ✉️ Asunto del .eml: Informe Veeam Backup - Olímpica S.A. 25 de Mayo 2025
    🕒 Hora de envío (EML Adjunto en COT): Mon 2025-05-26 11:03
    🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.

Backup a disco - veeam
Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
Cantidad MV respaldadas de Jobs ejecutados
200
Cantidad MV no respaldadas de los Jobs ejecutados
0
Hora de inicio de la ventana
25/5/2025 - 7:00 PM
Hora de finalización de la ventana
25/05/2025 - 10:29 PM
Tiempo de Backup
3h 29m
Backup a cinta - TAPE_OLIMPICA_NEW
  Observaciones
Porcentaje completado
98%
EN EJECUCIÓN
Porcentaje faltante
2%
TB respaldadas
65.7 TB
TB no respaldadas
0.8 TB
Hora de inicio
25/05/2025 - 1

    ✉️ Asunto del .eml: Informe Final Veeam Backup - Olímpica S.A. 24 de Mayo 2025
    🕒 Hora de envío (EML Adjunto en COT): Sun 2025-05-25 22:32
    🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.

Replication
Observaciones
Porcentaje completado
99%
FINALIZADO
Total de Jobs Programados
31
Jobs Replicados
30
Jobs No Replicados
1
Hora de inicio de la ventana
25/05/2025 - 1:00 AM
Hora de finalización de la ventana
25/05/2025 - 20:42 PM
Tiempo de backup
                              19h 42m



Quedamos atentos a sus comentarios.


Siéntase en libertad de comunicarse con nosotros a nuestra línea en Barranquilla tel. 3612752, a nuestra línea nacional 018000119966, o a nuestro correo electrónico servicio@redsis.com<mailto:servicio@redsis.com>, donde le brindaremos la información adicional que pueda requerir.



Es un gusto poder atenderle.





[cid:ecbee6da-8bfd-4219-a713-15019c2c8750]

C

    ✉️ Asunto del .eml: Informe Veeam Backup - Olímpica S.A. 23 de Mayo 2025
    🕒 Hora de envío (EML Adjunto en COT): Sun 2025-05-25 07:24
    🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.

Replication
Observaciones
Porcentaje completado
99%
FINALIZADO
Total de Jobs Programados
31
Jobs Replicados
30
Jobs No Replicados
1
Hora de inicio de la ventana
24/05/2025 - 1:00 AM
Hora de finalización de la ventana
         25/05/2025 - 12:54 AM
Tiempo de backup
                              23h 58m


Quedamos atentos a sus comentarios.


Siéntase en libertad de comunicarse con nosotros a nuestra línea en Barranquilla tel. 3612752, a nuestra línea nacional 018000119966, o a nuestro correo electrónico servicio@redsis.com<mailto:servicio@redsis.com>, donde le brindaremos la información adicional que pueda requerir.



Es un gusto poder atenderle.





[cid:33c63b97-40f8-4dd7-bad9-92dda98f5695]


...

    ✉️ Asunto del .eml: Informe Veeam Backup - Olímpica S.A. 3 de Mayo 2025
    🕒 Hora de envío (EML Adjunto en COT): Sun 2025-05-04 19:23
    🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.

Backup a disco - veeam
Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
Cantidad MV respaldadas de Jobs ejecutados
200
Cantidad MV no respaldadas de los Jobs ejecutados
0
Hora de inicio de la ventana
3/5/2025 - 7:00 PM
Hora de finalización de la ventana
         4/5/2025 - 1:11 AM
Tiempo de Backup
6h 11m
Backup a cinta - TAPE_OLIMPICA_NEW
  Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
TB respaldadas
65.7 TB
TB no respaldadas
0 TB
Hora de inicio
      4/5/2025 - 1:11 AM
Hora de finalización
4/5/2025 - 1:27 PM
Tiempo de Backup
          12h 15m
Estado
FINALIZADO
Replication
Observaciones
Porcentaje completado
100%
FINALIZADO
Total de Jobs Pro

    ✉️ Asunto del .eml: Informe Final Veeam Backup - Olímpica S.A. 2 de Mayo 2025
    🕒 Hora de envío (EML Adjunto en COT): Sun 2025-05-04 00:45
    🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.

Backup a disco - veeam
Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
Cantidad MV respaldadas de Jobs ejecutados
200
Cantidad MV no respaldadas de los Jobs ejecutados
0
Hora de inicio de la ventana
2/5/2025 - 7:00 PM
Hora de finalización de la ventana
         2/5/2025 - 11:48 PM
Tiempo de Backup
4h 48m
Backup a cinta - TAPE_OLIMPICA_NEW
  Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
TB respaldadas
65.6 TB
TB no respaldadas
0 TB
Hora de inicio
      2/5/2025 - 11:48 PM
Hora de finalización
3/5/2025 - 12:01 PM
Tiempo de Backup
          12h 12m
Estado
FINALIZADO
Replication
Observaciones
Porcentaje completado
99%
FINALIZADO
Total de 

    ✉️ Asunto del .eml: Informe Veeam Backup - Olímpica S.A. 2 de Mayo 2025
    🕒 Hora de envío (EML Adjunto en COT): Sat 2025-05-03 05:52
    🧾 Cuerpo del correo:
Cordial saludo, estimado cliente

El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.
Backup a disco - veeam
Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
Cantidad MV respaldadas de Jobs ejecutados
200
Cantidad MV no respaldadas de los Jobs ejecutados
0
Hora de inicio de la ventana
2/5/2025 - 7:00 PM
Hora de finalización de la ventana
         2/5/2025 - 11:48 PM
Tiempo de Backup
4h 48m
Backup a cinta - TAPE_OLIMPICA_MENSUAL
  Observaciones
Porcentaje completado
60%
EN EJECUCIÓN
Porcentaje faltante
40%
TB respaldadas
39.4 TB
TB no respaldadas
26.2 TB
Hora de inicio
      2/5/2025 - 11:48 PM
Hora de finalización

Tiempo de Backup
          5h 50m
Estado
RUNNING
Replication
Observaciones
Porcentaje completado
39%
EN EJECUCIÓN
Total de Jobs Programados
31


    ✉️ Asunto del .eml: Informe Veeam Backup - Olímpica S.A. 1 de Mayo 2025
    🕒 Hora de envío (EML Adjunto en COT): Fri 2025-05-02 06:32
    🧾 Cuerpo del correo:
Cordial saludo, estimado cliente


El siguiente es el resumen del reporte de la infraestructura de los Backups FULL con Veeam B&R.

Backup a disco - veeam
Observaciones
Porcentaje completado
100%
FINALIZADO
Porcentaje faltante
0%
Cantidad MV respaldadas de Jobs ejecutados
200
Cantidad MV no respaldadas de los Jobs ejecutados
0
Hora de inicio de la ventana
1/5/2025 - 7:00 PM
Hora de finalización de la ventana
                   2/5/2025 - 2:45 PM
Tiempo de Backup
7h 45m
Backup a cinta - TAPE_OLIMPICA_MENSUAL
  Observaciones
Porcentaje completado
83%
EN EJECUCIÓN
Porcentaje faltante
17%
TB respaldadas
42.6 TB
TB no respaldadas
8.3 TB
Hora de inicio
                   1/5/2025 - 11:51 PM
Hora de finalización

Tiempo de Backup
          6h 23m
Estado
RUNNING
Replication
Observaciones
Porcentaje completado
39%
EN EJECUCIÓN
Total 


Procesamiento de correos finalizado.


# PROCESAR TXT ULTIMA PRUEBA

EL siguiente codigo procesa bien el archivo de salida.txt pero no elimina duplicados de FInalizado 

# VERSION PARA QUE ELIMINE DUPLICADOS

In [20]:
import re
import pandas as pd
from datetime import datetime

def parse_log_and_export_to_excel(log_file, output_excel_file="informe_backups.xlsx"):
    """
    Parsea un archivo de log que contiene resúmenes de correos de backups y exporta
    la información relevante a un archivo Excel con múltiples hojas,
    manejando múltiples ocurrencias de backups por correo y deduplicando
    los registros 'FINALIZADO' por fecha y hora más reciente,
    basado en la hora de inicio de la ventana y las métricas de respaldo para identificar trabajos únicos.
    Además, organiza la salida final de cada hoja por las columnas "Fecha" y "Hora" de forma ascendente.

    Args:
        log_file (str): La ruta al archivo de log de entrada.
        output_excel_file (str): La ruta y nombre del archivo Excel de salida.
    """
    with open(log_file, 'r', encoding='utf-8') as f:
        content = f.read()

    # Dividir el contenido en bloques de correo
    email_blocks = re.split(r'--- Procesando correo ---', content)[1:]

    # Listas para almacenar los datos de cada tipo de backup
    backup_disco_data = []
    backup_cinta_data = []
    tape_mensual_data = []
    replication_data = []

    def parse_section_content(section_content, section_title):
        """
        Extrae las métricas y detalles de una sección de backup específica.
        """
        data = {}

        # Expresiones regulares para extraer los campos
        comp_match = re.search(r'Porcentaje completado\s*(\d+%)', section_content)
        data['Porcentaje completado'] = comp_match.group(1) if comp_match else 'N/A'

        # El estado se extrae de la misma línea que el porcentaje completado
        state_match = re.search(r'Porcentaje completado\s*\d+%\s*([A-Z\s]+)', section_content)
        data['Estado'] = state_match.group(1).strip() if state_match else 'N/A'

        falt_match = re.search(r'Porcentaje faltante\s*(\d+%)', section_content)
        data['Porcentaje faltante'] = falt_match.group(1) if falt_match else 'N/A'

        tb_resp_match = re.search(r'TB respaldadas\s*([\d\.]+\s*TB)', section_content)
        data['TB respaldadas'] = tb_resp_match.group(1) if tb_resp_match else 'N/A'
        tb_no_resp_match = re.search(r'TB no respaldadas\s*([\d\.]+\s*TB)', section_content)
        data['TB no respaldadas'] = tb_no_resp_match.group(1) if tb_no_resp_match else 'N/A'

        mv_resp_match = re.search(r'Cantidad MV respaldadas de Jobs ejecutados\s*(\d+)', section_content)
        data['Cantidad MV respaldadas de Jobs ejecutados'] = mv_resp_match.group(1) if mv_resp_match else 'N/A'
        mv_no_resp_match = re.search(r'Cantidad MV no respaldadas de los Jobs ejecutados\s*(\d+)', section_content)
        data['Cantidad MV no respaldadas de los Jobs ejecutados'] = mv_no_resp_match.group(1) if mv_no_resp_match else 'N/A'

        total_jobs_match = re.search(r'Total de Jobs Programados\s*(\d+)', section_content)
        data['Total de Jobs Programados'] = total_jobs_match.group(1) if total_jobs_match else 'N/A'
        jobs_rep_match = re.search(r'Jobs Replicados\s*(\d+)', section_content)
        data['Jobs Replicados'] = jobs_rep_match.group(1) if jobs_rep_match else 'N/A'
        jobs_no_rep_match = re.search(r'Jobs No Replicados\s*(\d+)', section_content)
        data['Jobs No Replicados'] = jobs_no_rep_match.group(1) if jobs_no_rep_match else 'N/A'

        # Extraer Fecha y Hora de inicio/finalización de la ventana
        # Usamos (?: de la ventana)? para hacer " de la ventana" opcional
        hora_inicio_match = re.search(r'Hora de inicio(?: de la ventana)?\s*(\d{1,2}/\d{1,2}/\d{4} - \d{1,2}:\d{2} (?:AM|PM))', section_content)
        hora_finalizacion_match = re.search(r'Hora de finalización(?: de la ventana)?\s*(\s*\d{1,2}/\d{1,2}/\d{4} - \d{1,2}:\d{2} (?:AM|PM))?', section_content)
        tiempo_backup_match = re.search(r'Tiempo de Backup\s*(\d+h \d+m)', section_content)

        data['Fecha de inicio de la ventana'] = "N/A"
        data['Hora de inicio de la ventana'] = "N/A"
        data['Fecha Real_Section'] = "N/A" # Variable auxiliar para la fecha real del mail si no hay en la sección
        if hora_inicio_match:
            try:
                # Parsear la fecha y hora completa
                dt_object = datetime.strptime(hora_inicio_match.group(1), '%d/%m/%Y - %I:%M %p')
                data['Fecha de inicio de la ventana'] = dt_object.strftime('%d/%m/%Y')
                data['Hora de inicio de la ventana'] = dt_object.strftime('%I:%M %p')
                data['Fecha Real_Section'] = dt_object.strftime('%d-%b') # Formato para la columna 'Fecha Real'
            except ValueError:
                pass # Si falla el parseo, se queda en 'N/A'

        data['Fecha de finalización de la ventana'] = "N/A"
        data['Hora de finalización de la ventana'] = "N/A"
        
        if hora_finalizacion_match: 
            final_time_str = hora_finalizacion_match.group(1)
            if final_time_str and final_time_str.strip(): # Asegurarse de que no sea solo espacio en blanco
                try:
                    dt_object = datetime.strptime(final_time_str.strip(), '%d/%m/%Y - %I:%M %p')
                    data['Fecha de finalización de la ventana'] = dt_object.strftime('%d/%m/%Y') 
                    data['Hora de finalización de la ventana'] = dt_object.strftime('%I:%M %p') 
                except ValueError:
                    pass

        data['Tiempo de Backup'] = tiempo_backup_match.group(1) if tiempo_backup_match else 'N/A'

        return data

    for block in email_blocks:
        # Extraer la hora de envío del correo principal (EML Adjunto)
        hora_envio_eml_match = re.search(r'🕒 Hora de envío \(EML Adjunto en COT\): (\w{3} \d{4}-\d{2}-\d{2} \d{2}:\d{2})', block)

        fecha_eml_formato_dia_mes = "N/A"
        hora_eml = "N/A"
        
        if hora_envio_eml_match:
            try:
                date_time_str = hora_envio_eml_match.group(1)
                # El formato es "Day Month-DD-YY HH:MM", necesitamos YYYY-MM-DD HH:MM para un parseo robusto
                # Para Fecha (DD-Mon) y Hora (HH:MM), extraemos directamente
                dt_object = datetime.strptime(date_time_str.split(' ')[1] + ' ' + date_time_str.split(' ')[2], '%Y-%m-%d %H:%M')
                fecha_eml_formato_dia_mes = dt_object.strftime('%d-%b') # Formato "01-May"
                hora_eml = dt_object.strftime('%H:%M') # Hora en formato 24h
            except ValueError:
                pass

        # Buscar las secciones de backup dentro de cada bloque de correo
        section_pattern = re.compile(
            r'(Backup a disco - veeam|Backup a cinta - TAPE_OLIMPICA_NEW|Backup a cinta - TAPE_OLIMPICA_MENSUAL|Replication)\s*Observaciones\s*(.*?)(?=\n(?:Backup a disco|Backup a cinta|Replication|Quedamos atentos a sus comentarios|“AVISO LEGAL|\Z))',
            re.DOTALL # Permite que . coincida con saltos de línea
        )
        
        all_sections_in_block = section_pattern.findall(block)

        # Iterar sobre las secciones encontradas
        for section_title_found, section_content_found in all_sections_in_block:
            metrics = parse_section_content(section_content_found, section_title_found)
            
            # Datos comunes a todos los tipos de backup
            common_data = {
                'Fecha Real': metrics.get('Fecha Real_Section', fecha_eml_formato_dia_mes), # Prioriza la fecha de la sección, sino usa la del correo
                'Fecha': fecha_eml_formato_dia_mes, # Esta es la columna para el ordenamiento final
                'Hora': hora_eml, # Esta es la columna para el ordenamiento final
                'Porcentaje completado': metrics.get('Porcentaje completado'),
                'Porcentaje faltante': metrics.get('Porcentaje faltante'),
                'Fecha de inicio de la ventana': metrics.get('Fecha de inicio de la ventana'),
                'Hora de inicio de la ventana': metrics.get('Hora de inicio de la ventana'),
                'Fecha de finalización de la ventana': metrics.get('Fecha de finalización de la ventana'),
                'Hora de finalización de la ventana': metrics.get('Hora de finalización de la ventana'),
                'Tiempo de Backup': metrics.get('Tiempo de Backup'),
                'Estado': metrics.get('Estado'),
            }

            # Añadir datos específicos de cada tipo de backup y el nombre de la actividad
            if "Backup a disco - veeam" in section_title_found:
                if common_data.get('Porcentaje completado') != 'N/A':
                    backup_disco_data.append({
                        **common_data,
                        'Cantidad MV respaldadas de Jobs ejecutados': metrics.get('Cantidad MV respaldadas de Jobs ejecutados'),
                        'Cantidad MV no respaldadas de los Jobs ejecutados': metrics.get('Cantidad MV no respaldadas de los Jobs ejecutados'),
                        'Actividad': 'Backup a disco - Veeam'
                    })
            elif "Backup a cinta - TAPE_OLIMPICA_NEW" in section_title_found:
                if common_data.get('Porcentaje completado') != 'N/A':
                    backup_cinta_data.append({
                        **common_data,
                        'TB respaldadas': metrics.get('TB respaldadas'),
                        'TB no respaldadas': metrics.get('TB no respaldadas'),
                        'Cantidad MV respaldadas de Jobs ejecutados': 'N/A',
                        'Cantidad MV no respaldadas de los Jobs ejecutados': 'N/A',
                        'Actividad': 'Backup a cinta - TAPE_DIARIO_NEW'
                    })
            elif "Backup a cinta - TAPE_OLIMPICA_MENSUAL" in section_title_found:
                if common_data.get('Porcentaje completado') != 'N/A':
                    tape_mensual_data.append({
                        **common_data,
                        'TB respaldadas': metrics.get('TB respaldadas'),
                        'TB no respaldadas': metrics.get('TB no respaldadas'),
                        'Cantidad MV respaldadas de Jobs ejecutados': 'N/A',
                        'Cantidad MV no respaldadas de los Jobs ejecutados': 'N/A',
                        'Actividad': 'Backup a cinta - TAPE_OLIMPICA_MENSUAL'
                    })
            elif "Replication" in section_title_found:
                if common_data.get('Porcentaje completado') != 'N/A':
                    replication_data.append({
                        **common_data,
                        'Porcentaje faltante': 'N/A',
                        'Total de Jobs Programados': metrics.get('Total de Jobs Programados'),
                        'Jobs Replicados': metrics.get('Jobs Replicados'),
                        'Jobs No Replicados': metrics.get('Jobs No Replicados'),
                        'Cantidad MV respaldadas de Jobs ejecutados': 'N/A',
                        'Cantidad MV no respaldadas de los Jobs ejecutados': 'N/A',
                        'Actividad': 'Replication'
                    })

    # Crear DataFrames a partir de los datos recolectados
    df_backup_disco = pd.DataFrame(backup_disco_data)
    df_backup_cinta = pd.DataFrame(backup_cinta_data)
    df_tape_mensual = pd.DataFrame(tape_mensual_data)
    df_replication = pd.DataFrame(replication_data)

    # Función auxiliar para parsear fechas y horas en formato de fecha y hora para Pandas
    def parse_datetime_cols(df, date_col, time_col, date_format, time_format):
        combined_strings = pd.Series([None] * len(df), dtype='object') # Initialize with None
        
        # Determine if we need to add the year based on the date_format
        if date_format == '%d-%b': # For 'Fecha Real' and 'Fecha' (DD-Mon format)
            current_year = datetime.now().year
            temp_date_col = df[date_col].apply(lambda x: f"{x}-{current_year}" if x != 'N/A' else x)
            format_str = f"{date_format}-%Y {time_format}"
        elif date_format == '%d/%m/%Y': # For 'Fecha de inicio/finalización de la ventana' (DD/MM/YYYY format)
            temp_date_col = df[date_col]
            format_str = f"{date_format} {time_format}"
        else: # Fallback or other formats if needed
            temp_date_col = df[date_col]
            format_str = f"{date_format} {time_format}" # Default to not adding year

        # Combine strings only for non-N/A entries
        valid_indices = (df[date_col] != 'N/A') & (df[time_col] != 'N/A')
        
        if valid_indices.any():
            combined_strings.loc[valid_indices] = temp_date_col[valid_indices].astype(str) + ' ' + df[time_col][valid_indices].astype(str)

        return pd.to_datetime(combined_strings, format=format_str, errors='coerce')


    # --- Lógica de deduplicación y ordenamiento final: Aplicar a cada DataFrame individualmente ---
    for df_name, df_obj in [('df_backup_disco', df_backup_disco), 
                             ('df_backup_cinta', df_backup_cinta), 
                             ('df_tape_mensual', df_tape_mensual), 
                             ('df_replication', df_replication)]:

        if not df_obj.empty: # Solo procesar si el DataFrame no está vacío
            # Crear columnas de datetime combinadas para ordenar los registros 'FINALIZADO'
            # Se prioriza la fecha de finalización, luego la de inicio de ventana, y finalmente la del log.
            df_obj['Fin_Ventana_dt'] = parse_datetime_cols(df_obj, 'Fecha de finalización de la ventana', 'Hora de finalización de la ventana', '%d/%m/%Y', '%I:%M %p')
            df_obj['Inicio_Ventana_dt'] = parse_datetime_cols(df_obj, 'Fecha de inicio de la ventana', 'Hora de inicio de la ventana', '%d/%m/%Y', '%I:%M %p')
            # Para la deduplicación de 'FINALIZADO', también usamos la Fecha/Hora del correo como fallback
            df_obj['Log_Correo_dt'] = parse_datetime_cols(df_obj, 'Fecha', 'Hora', '%d-%b', '%H:%M')


            # Separar los registros "FINALIZADO" de los demás estados
            df_finalizado = df_obj[df_obj['Estado'].str.contains('FINALIZADO', na=False)]
            df_otros = df_obj[~df_obj['Estado'].str.contains('FINALIZADO', na=False)]

            if not df_finalizado.empty:
                # Definir la columna 'Sort_Deduplication_Datetime' para ordenar. Se usa el primero disponible.
                # Priorizamos Fin_Ventana_dt > Inicio_Ventana_dt > Log_Correo_dt
                df_finalizado['Sort_Deduplication_Datetime'] = df_finalizado['Fin_Ventana_dt'].fillna(df_finalizado['Inicio_Ventana_dt']).fillna(df_finalizado['Log_Correo_dt'])
                
                # Definir el subconjunto de columnas para la deduplicación,
                # incluyendo las métricas relevantes para cada tipo de backup.
                dedup_subset = ['Actividad', 'Fecha de inicio de la ventana', 'Hora de inicio de la ventana']
                
                if df_name == 'df_backup_disco':
                    dedup_subset.extend(['Cantidad MV respaldadas de Jobs ejecutados', 'Cantidad MV no respaldadas de los Jobs ejecutados'])
                elif df_name == 'df_backup_cinta' or df_name == 'df_tape_mensual':
                    dedup_subset.extend(['TB respaldadas', 'TB no respaldadas'])
                elif df_name == 'df_replication':
                    dedup_subset.extend(['Total de Jobs Programados', 'Jobs Replicados', 'Jobs No Replicados'])

                # Ordenar por 'Sort_Deduplication_Datetime' ascendente y eliminar duplicados
                df_finalizado_deduplicated = df_finalizado.sort_values(by='Sort_Deduplication_Datetime', ascending=True, na_position='last') \
                                                          .drop_duplicates(subset=dedup_subset, keep='first')
                
                # Unir de nuevo los registros deduplicados con los "otros" (no finalizados)
                df_obj = pd.concat([df_finalizado_deduplicated, df_otros]).reset_index(drop=True)
            
            # Eliminar las columnas temporales de datetime usadas para deduplicación
            df_obj = df_obj.drop(columns=['Fin_Ventana_dt', 'Inicio_Ventana_dt', 'Log_Correo_dt', 'Sort_Deduplication_Datetime'], errors='ignore')

            # --- Ordenamiento final para toda la hoja por 'Fecha' y 'Hora' ---
            df_obj['Final_Sheet_Sort_Datetime'] = parse_datetime_cols(df_obj, 'Fecha', 'Hora', '%d-%b', '%H:%M')
            df_obj = df_obj.sort_values(by='Final_Sheet_Sort_Datetime', ascending=True, na_position='last')
            df_obj = df_obj.drop(columns=['Final_Sheet_Sort_Datetime'], errors='ignore') # Eliminar la columna temporal de ordenamiento

            # Reasignar el DataFrame modificado a su variable original
            if df_name == 'df_backup_disco':
                df_backup_disco = df_obj
            elif df_name == 'df_backup_cinta':
                df_backup_cinta = df_obj
            elif df_name == 'df_tape_mensual':
                df_tape_mensual = df_obj
            elif df_name == 'df_replication':
                df_replication = df_obj

    # Definir el orden de las columnas para cada hoja del Excel
    cols_backup_disco = ['Fecha Real', 'Fecha', 'Hora', 'Porcentaje completado', 'Porcentaje faltante',
                         'Cantidad MV respaldadas de Jobs ejecutados', 'Cantidad MV no respaldadas de los Jobs ejecutados',
                         'Fecha de inicio de la ventana', 'Hora de inicio de la ventana',
                         'Fecha de finalización de la ventana', 'Hora de finalización de la ventana',
                         'Tiempo de Backup', 'Estado', 'Actividad']

    cols_backup_cinta = ['Fecha Real', 'Fecha', 'Hora', 'Porcentaje completado', 'Porcentaje faltante',
                         'TB respaldadas', 'TB no respaldadas',
                         'Fecha de inicio de la ventana', 'Hora de inicio de la ventana',
                         'Fecha de finalización de la ventana', 'Hora de finalización de la ventana',
                         'Tiempo de Backup', 'Estado', 'Actividad']

    cols_tape_mensual = ['Fecha Real', 'Fecha', 'Hora', 'Porcentaje completado', 'Porcentaje faltante',
                         'TB respaldadas', 'TB no respaldadas',
                         'Fecha de inicio de la ventana', 'Hora de inicio de la ventana',
                         'Fecha de finalización de la ventana', 'Hora de finalización de la ventana',
                         'Tiempo de Backup', 'Estado', 'Actividad']

    cols_replication = ['Fecha Real', 'Fecha', 'Hora', 'Porcentaje completado',
                        'Total de Jobs Programados', 'Jobs Replicados', 'Jobs No Replicados',
                        'Fecha de inicio de la ventana', 'Hora de inicio de la ventana',
                        'Fecha de finalización de la ventana', 'Hora de finalización de la ventana',
                        'Estado', 'Actividad']

    def reorder_and_fill_missing_cols(df, columns):
        """
        Reordena las columnas de un DataFrame y añade las que falten con 'N/A'.
        """
        for col in columns:
            if col not in df.columns:
                df[col] = 'N/A' # Añadir columna si no existe
        return df[columns]

    # Aplicar el reordenamiento y relleno de columnas a cada DataFrame
    if not df_backup_disco.empty:
        df_backup_disco = reorder_and_fill_missing_cols(df_backup_disco, cols_backup_disco)
    if not df_backup_cinta.empty:
        df_backup_cinta = reorder_and_fill_missing_cols(df_backup_cinta, cols_backup_cinta)
    if not df_tape_mensual.empty:
        df_tape_mensual = reorder_and_fill_missing_cols(df_tape_mensual, cols_tape_mensual)
    if not df_replication.empty:
        df_replication = reorder_and_fill_missing_cols(df_replication, cols_replication)

    # Exportar los DataFrames a diferentes hojas de un archivo Excel
    with pd.ExcelWriter(output_excel_file, engine='xlsxwriter') as writer:
        if not df_backup_disco.empty:
            df_backup_disco.to_excel(writer, sheet_name='Backup a Disco', index=False)
        if not df_backup_cinta.empty:
            df_backup_cinta.to_excel(writer, sheet_name='Backup a cinta', index=False)
        if not df_tape_mensual.empty:
            df_tape_mensual.to_excel(writer, sheet_name='Tape Mensual', index=False)
        if not df_replication.empty:
            df_replication.to_excel(writer, sheet_name='Replication', index=False)

    print(f"Datos extraídos y guardados en '{output_excel_file}' con las hojas: Backup a Disco, Backup a cinta, Tape Mensual, Replication.")

# --- Ejecutar la extracción y exportación ---
# Asegúrate de que el archivo 'salida.txt' esté en la misma ubicación que tu script
# o proporciona la ruta completa al archivo.
parse_log_and_export_to_excel('salida.txt', 'informe_backups.xlsx')

/tmp/ipykernel_6788/3818812907.py:240: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_finalizado['Sort_Deduplication_Datetime'] = df_finalizado['Fin_Ventana_dt'].fillna(df_finalizado['Inicio_Ventana_dt']).fillna(df_finalizado['Log_Correo_dt'])
/tmp/ipykernel_6788/3818812907.py:240: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_finalizado['Sort_Deduplication_Datetime'] = df_finalizado['Fin_Ventana_dt'].fillna(df_finalizado['Inicio_Ventana_dt']).fillna(df_finalizado['Log_Correo_dt'])
/tmp/ipykernel_6788/

Datos extraídos y guardados en 'informe_backups.xlsx' con las hojas: Backup a Disco, Backup a cinta, Tape Mensual, Replication.
